In [250]:
import pandas as pd
import numpy as np

先確保每個欄位都沒有奇怪data

1. 先手動刪除除了這七個之外的欄位的資料
2. 檢查每個欄位的值是否都在domain裡面
    * space: 1, 2, 3, 4, 5, 6, 7, 8, 9, FZ, L, M, Y
    * action: JS, SS, FS, R, G, A, B, D, OP(對方問題)

In [251]:
df_IRI = pd.read_excel('Data\IRI VS POL.xlsx', sheet_name = 'IRI Set5')
df_IRI = df_IRI.dropna(how='all')
df_IRI.insert(1, 'Team', 'IRI')
df_IRI.insert(2, 'Rally', np.nan)
df_IRI.head()

,Is_blue_space,Team,Rally,No.,Space,Action,Errors,Attempts,Score
0,NaN,IRI,NaN,2.0,Y,JS,NaN,NaN,NaN
1,0.0,IRI,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,IRI,NaN,1.0,1,B,NaN,NaN,NaN
3,NaN,IRI,NaN,18.0,2,D,NaN,NaN,NaN
4,NaN,IRI,NaN,1.0,1,G,NaN,NaN,NaN


In [252]:
pd.isna(df_IRI.iloc[0]['Is_blue_space'])

True

In [253]:
df_POL = pd.read_excel('Data\IRI VS POL.xlsx', sheet_name = 'POL Set5')
df_POL = df_POL.dropna(how='all')
df_POL.insert(1, 'Team', 'POL')
df_POL.insert(2, 'Rally', np.nan)
df_POL.head()

,Is_blue_space,Team,Rally,No.,Space,Action,Errors,Attempts,Score
0,1.0,POL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,POL,NaN,17.0,5,R,NaN,NaN,NaN
2,NaN,POL,NaN,19.0,2,G,NaN,NaN,NaN
3,NaN,POL,NaN,6.0,3,A,NaN,NaN,NaN
4,0.0,POL,NaN,NaN,NaN,NaN,NaN,NaN,NaN


合併兩個dataset
* df_1: 發球方
* df_2: 另一方
* i, j 分別代表兩邊目前的指標(?
---
要加入的欄位：rally_num, team


In [271]:
def get_rally_sequence(df_1, df_2, i, j, rally):
    df_combine = pd.DataFrame(columns=df_1.columns)
    # 發球方發球
    df_combine = df_combine.append(df_1.iloc[i])
    i += 1
    alternate = 1
    while(i < len(df_1)-1 and j < len(df_2)-1):
        print()
        if(alternate == 0): # df_1
            if(not pd.isna(df_1.iloc[i]['Is_blue_space'])):
                i += 1
            df_combine = df_combine.append(df_1.iloc[i])
            i += 1
            while(pd.isna(df_1.iloc[i]['Is_blue_space']) and 'S' not in str(df_1.iloc[i]['Action'])): # 遇到空格或再次發球前
                print('i: ' + str(i))
                df_combine = df_combine.append(df_1.iloc[i])
                i += 1
                if(i == len(df_1)):
                    break

            # 因為空格而停止
            if(not pd.isna(df_1.iloc[i]['Is_blue_space'])):
                if(df_1.iloc[i]['Is_blue_space'] == 1):
                    break
                else:
                    alternate = 1
                    continue

            # 因為再次發球而停止
            if('S' in df_1.iloc[i]['Action']):
                # 先確認另一隊在藍色空格之前的state是否全部記錄到
                while(df_2.iloc[j]['Is_blue_space'] != 1 and j < len(df_2)):
                    if(df_2.iloc[j]['Is_blue_space'] != 0):
                        df_combine = df_combine.append(df_2.iloc[j])
                    j += 1
                break

        elif(alternate == 1):   # df_2
            if(not pd.isna(df_2.iloc[j]['Is_blue_space'])):
                j += 1
            df_combine = df_combine.append(df_2.iloc[j])
            j += 1
            while(pd.isna(df_2.iloc[j]['Is_blue_space']) and 'S' not in df_2.iloc[j]['Action']): # 遇到空格或再次發球前
                print('j: ' + str(j))
                df_combine = df_combine.append(df_2.iloc[j])
                j += 1
                if(j == len(df_2)):
                    break

            if(not pd.isna(df_2.iloc[j]['Is_blue_space'])): # is space
                if(df_2.iloc[j]['Is_blue_space'] == 1): # 失誤
                    break
                else:
                    alternate = 0
                    continue
            
            if('S' in df_2.iloc[j]['Action']):  # 再次發球 -> 下一round
                # 先確認另一隊在藍色空格之前的state是否全部記錄到
                while(df_1.iloc[i]['Is_blue_space'] != 1 and i < len(df_1)):
                    if(df_1.iloc[i]['Is_blue_space'] != 0):
                        df_combine = df_combine.append(df_1.iloc[i])
                    i += 1
                break
    if(i == len(df_1)):
        while(j != len(df_2)):
            df_combine = df_combine.append(df_2.iloc[j])
            j += 1
    if(j == len(df_2)):
        while(i != len(df_1)):
            df_combine = df_combine.append(df_1.iloc[i])
            i += 1

    df_combine['Rally'] = rally
    return df_combine, i, j

In [272]:
df_total_game = pd.DataFrame(columns=df_IRI.columns)

In [273]:
i = 0
j = 0
rally_num = 0
while(i < len(df_IRI) or j < len(df_POL)):
    print('\nrally: ' + str(rally_num) + '\n-------------------')
    if(df_IRI.iloc[i]['Is_blue_space'] == 1):   # 對方發球
        df_combine, j, i = get_rally_sequence(df_POL, df_IRI, j, i, rally_num)
    else:
        df_combine, i, j = get_rally_sequence(df_IRI, df_POL, i, j, rally_num)
    print(i, j)
    df_total_game = pd.concat([df_total_game, df_combine])
    rally_num += 1


rally: 0
-------------------

j: 2
j: 3

i: 3
i: 4
i: 5

j: 6
j: 7
j: 8


j: 11
j: 12
10 13

rally: 1
-------------------

j: 12
j: 13
14 14

rally: 2
-------------------

j: 16
j: 17

20 16

rally: 3
-------------------

j: 22
j: 23

28 19

rally: 4
-------------------

j: 30
j: 31
32 29

rally: 5
-------------------

j: 31
j: 32
35 33

rally: 6
-------------------

37 34

rally: 7
-------------------

j: 36
j: 37
40 38

rally: 8
-------------------


i: 41
i: 42
44 43

rally: 9
-------------------

46 44

rally: 10
-------------------

j: 46
j: 47
49 48

rally: 11
-------------------

j: 51
j: 52
53 51

rally: 12
-------------------

54 53

rally: 13
-------------------

j: 56
j: 57

i: 56
i: 57


i: 60
i: 61
62 62

rally: 14
-------------------

j: 64
j: 65
66 65

rally: 15
-------------------

j: 67
j: 68
69 69

rally: 16
-------------------

j: 71
72 70

rally: 17
-------------------

j: 74
j: 75

78 72

rally: 18
-------------------

j: 80
j: 81
82 73

rally: 19
----------------

In [275]:
df_total_game.head(20)

,Is_blue_space,Team,Rally,No.,Space,Action,Errors,Attempts,Score
0,NaN,IRI,0,2.0,Y,JS,NaN,NaN,NaN
1,NaN,POL,0,17.0,5,R,NaN,NaN,NaN
2,NaN,POL,0,19.0,2,G,NaN,NaN,NaN
3,NaN,POL,0,6.0,3,A,NaN,NaN,NaN
2,NaN,IRI,0,1.0,1,B,NaN,NaN,NaN
3,NaN,IRI,0,18.0,2,D,NaN,NaN,NaN
4,NaN,IRI,0,1.0,1,G,NaN,NaN,NaN
5,NaN,IRI,0,22.0,1,A,NaN,NaN,NaN
5,NaN,POL,0,7.0,3,B,NaN,NaN,NaN
6,NaN,POL,0,14.0,7,D,NaN,1.0,NaN


In [ ]:
for idx, row in df_IRI.iterrows():
    if(idx < 5):
        d = d.append(d.iloc[idx])

In [ ]:
d.iloc[1]['Is_blue_space']

0.0